---

# **Computer Vision (CV) for Human Action Recognition (HAR)**

---

<br />

Work in progress! This is my current work for my computer vision (cv) for human action recognition (HAR) project. My goal is to use use transfer learning with multiple state of the art computer vision models, tested on ImageNet, and apply them to my specific HAR application. This is to test whether the speed of training from transfer learning outweights the possible biases that can be found within the predefined weights of these models. To keep the testing simple, the trainable layers being added on are only dense and dropout layers. For these trainable layers, I am also using Keras Tuner to explore the full potential that these fully adapted models can obtain. Enjoy the code!

<br />

---

## Imports and System Setup

In [ ]:
# Set working environment variable (keep one true)
my_machine = False
colab = True
# Set TPU bool (can be used in colab)
# Current versioning error using a TPU so keep false (to be fixed)
tpu = False

# Assumed packages installed on local machine
# If colab install keras tuner (not included in colab)
if colab:
    %pip install -q -U keras-tuner

# Full imports
import os
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib as mp
import keras_tuner

# Partial imports
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.backend import batch_normalization
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Confirm completion
print( "Everything successfully imported!" )

In [ ]:
# If on your machine check for GPU and and enable memory growth
# Do not need to run if not using a GPU
if not tpu:
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    if not physical_devices:
        print( "No GPU recognized!" )
    else:
        print( "Number of GPUs recognized: ", len(physical_devices) )
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

# If using Colab you can connect and init a TPU for training
if tpu and colab:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tpu_stat = tf.distribute.experimental.TPUStrategy(tpu)

## Data Preprocessing

In [ ]:
# Collect train and test data from csv files
# HAR data found on Kaggle
if my_machine:
    ROOT_DIR = os.getcwd()
    train_set_csv = pd.read_csv( os.path.join(ROOT_DIR, "Training_set.csv") )
    test_set_csv = pd.read_csv( os.path.join(ROOT_DIR, "Testing_set.csv") )

if colab:
    from google.colab import files
    upload = files.upload()
    train_set_csv = pd.read_csv('Training_set.csv')
    test_set_csv = pd.read_csv('Testing_set.csv')

num_classes = len(train_set_csv['label'].unique())
print( "Total number of action classes: ", num_classes )
print( "List of action classes: " )
print( train_set_csv['label'].unique() )

In [ ]:
# Get path to or upload train and test data
if my_machine:
    TRAIN_DIR = os.path.join(ROOT_DIR, "train")
    TEST_DIR = os.path.join(ROOT_DIR, "test")
if colab:
    from google.colab import files
    upload = files.upload()
    with zipfile.ZipFile("train.zip", "r") as zip_ref:
        zip_ref.extractall()
    TRAIN_DIR = "train"
    with zipfile.ZipFile("test.zip", "r") as zip_ref:
        zip_ref.extractall()
    TEST_DIR = "test"

# Create train and validation datasets and apply transformations
train_val_image_generator = ImageDataGenerator(
    validation_split=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
)
train_dataset = train_val_image_generator.flow_from_dataframe(
    dataframe=train_set_csv,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='label',
    subset='training',
    shuffle=True,
    target_size=(224, 224),
    class_mode='categorical',
)
val_dataset = train_val_image_generator.flow_from_dataframe(
    dataframe=train_set_csv,
    directory=TRAIN_DIR,
    x_col='filename',
    y_col='label',
    subset='validation',
    shuffle=True,
    target_size=(224, 224),
    class_mode='categorical',
)

# Create test dataset and apply transformations
test_image_generator = ImageDataGenerator()
test_dataset = test_image_generator.flow_from_dataframe(
    dataframe=test_set_csv, 
    directory=TEST_DIR,
    x_col='filename',
    y_col=None,
    shuffle=False,
    target_size=(224, 224),
    class_mode=None
)

## VGG Model Build and Tuning

In [ ]:
def build_vgg_model(hp):
    # Create base cnn layers from VGG16 but cut off dense layers to use our own instead
    # Use imagenet weights and lock them to be untrainable
    vgg = Sequential()
    pretrained_model= tf.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False
    # Add dense layers to VGG16 for our personal use with 15 classes output
    # Use keras tuner for number of internal dense layers and number of nodes for each dense layer
    vgg.add( pretrained_model )
    vgg.add( Dropout(0.5) )
    vgg.add( Flatten() )
    vgg.add( Dense(512, activation='relu') )
    # Tunable layers
    vgg.add( Dropout(hp.Float("dropout_1", min_value=0.0, max_value=0.3, step=0.1)) )
    vgg.add( Dense(units=hp.Int("units_2", min_value=128, max_value=256, step=128), activation='relu') )
    vgg.add( Dropout(hp.Float("dropout_2", min_value=0.0, max_value=0.3, step=0.1)) )
    vgg.add( Dense(units=hp.Int("units_3", min_value=64, max_value=128, step=64), activation='relu') )
    vgg.add( Dropout(hp.Float("dropout_3", min_value=0.0, max_value=0.3, step=0.1)) )
    # Output layer
    vgg.add( Dense(15, activation='softmax') )
    # Compile model and return
    vgg.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return vgg

In [ ]:
# Define tuner search and check summary
tuner = keras_tuner.RandomSearch(
    hypermodel=build_vgg_model,
    objective="val_accuracy",
    overwrite=True,
    directory="/",
    project_name="vgg_for_har",
)
tuner.search_space_summary()

In [ ]:
# Run tuner search and get the optimal hyperparamters
tuner.search(x=train_dataset, validation_data=val_dataset, epochs=5)
tuned_hp = tuner.get_best_hyperparameters()[0]

# Check for tpu and then build optimally tuned model
if not tpu:
    vgg = build_vgg_model(tuned_hp)
if tpu and colab:
    with tpu_stat.scope():
        vgg = build_vgg_model(tuned_hp)

# Show summary of optimal model
vgg.summary()

## VGG Training

In [ ]:
# Set callback function for early stopping to save best epoch
callbacks = [
    ModelCheckpoint(filepath='best_effnet_har_model.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.1, min_lr=1.0e-7),
    EarlyStopping(monitor='val_accuracy', patience=10),
]

# Fit tuned model and save resulting weights
history = vgg.fit_generator(generator=train_dataset, validation_data=val_dataset, epochs=100, callbacks=callbacks)
vgg.save("vgg_har_model.h5")
vgg.save_weights("vgg_har_weights.h5")

# If using Colab it will save weights to local machine
if colab:
    from google.colab import files
    files.download("vgg_har_model.h5")
    files.download("vgg_har_weights.h5")

## EfficientNet Model Build and Tuning

In [ ]:
def build_effnet_model(hp):
    # Create base cnn layers from EfficientNet V2M but cut off dense layers to use our own instead
    # Use imagenet weights and lock them to be untrainable
    effnet = Sequential()
    pretrained_model= tf.keras.applications.EfficientNetV2M(include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False
    # Add dense layers to EfficientNet V2M for our personal use with 15 classes output
    # Use keras tuner for number of internal dense layers and number of nodes for each dense layer
    effnet.add( pretrained_model )
    effnet.add( Flatten() )
    effnet.add( Dense(512, activation='relu') )
    effnet.add( Dropout(0.5) )
    effnet.add( Dense(units=hp.Int("units_0", min_value=128, max_value=256, step=128), activation='relu') )
    dropout = hp.Boolean("dropout_0")
    if dropout:
        effnet.add( Dropout(0.5) )
    effnet.add( Dense(units=hp.Int("units_1", min_value=64, max_value=128, step=64), activation='relu') )
    dropout = hp.Boolean("dropout_1")
    if dropout:
        effnet.add( Dropout(0.5) )
    # Output layer
    effnet.add( Dense(15, activation='softmax') )
    # Compile model and return
    effnet.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return effnet

In [ ]:
# Define tuner search and check summary
tuner = keras_tuner.RandomSearch(
    hypermodel=build_effnet_model,
    objective="val_accuracy",
    max_trials=16,
    overwrite=True,
    directory="/",
    project_name="effnet_for_har",
)
tuner.search_space_summary()

In [ ]:
# Run tuner search and get the optimal hyperparamters
tuner.search(x=train_dataset, validation_data=val_dataset, epochs=10)
tuned_hp = tuner.get_best_hyperparameters()[0]

# Check for tpu and then build optimally tuned model
if not tpu:
    effnet = build_effnet_model(tuned_hp)
if tpu and colab:
    with tpu_stat.scope():
        effnet = build_effnet_model(tuned_hp)

# Show summary of optimal model
effnet.summary()

## EfficientNet Training

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Set callback function for early stopping to save best epoch
callbacks = [
    ModelCheckpoint(filepath='best_effnet_har_model.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.1, min_lr=1.0e-7),
    EarlyStopping(monitor='val_accuracy', patience=10),
]

# Fit tuned model and save resulting weights
history = effnet.fit_generator(generator=train_dataset, validation_data=val_dataset, epochs=100, callbacks=callbacks)
effnet.save("effnet_har_model.h5")
effnet.save_weights("effnet_har_weights.h5")

# If using Colab it will save weights to local machine
if colab:
    from google.colab import files
    files.download("effnet_har_model.h5")
    files.download("effnet_har_weights.h5")

## DenseNet Model Build and Tuning

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
def build_densenet_model(hp):
    # Create base cnn layers from DenseNet but cut off dense layers to use our own instead
    # Use imagenet weights and lock them to be untrainable
    densenet = Sequential()
    pretrained_model= tf.keras.applications.DenseNet201(include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False
    # Add dense layers to DenseNet for our personal use with 15 classes output
    # Use keras tuner for number of internal dense layers and number of nodes for each dense layer
    densenet.add( pretrained_model )
    densenet.add( Flatten() )
    densenet.add( Dense(512, activation='relu') )
    densenet.add( Dropout(0.5) )
    """
    densenet.add( Dense(units=hp.Int("units_0", min_value=128, max_value=256, step=128), activation='relu') )
    dropout = hp.Boolean("dropout_0")
    if dropout:
        densenet.add( Dropout(0.5) )
    densenet.add( Dense(units=hp.Int("units_1", min_value=64, max_value=128, step=64), activation='relu') )
    dropout = hp.Boolean("dropout_1")
    if dropout:
        densenet.add( Dropout(0.5) )
    """
    densenet.add( Dense(256, activation='relu') )
    # Output layer
    densenet.add( Dense(15, activation='softmax') )
    # Compile model and return
    densenet.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return densenet

In [ ]:
# Define tuner search and check summary
tuner = keras_tuner.RandomSearch(
    hypermodel=build_densenet_model,
    objective="val_accuracy",
    max_trials=16,
    overwrite=True,
    directory="/",
    project_name="densenet_for_har",
)
tuner.search_space_summary()

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Run tuner search and get the optimal hyperparamters
"""
tuner.search(x=train_dataset, validation_data=val_dataset, epochs=10)
tuned_hp = tuner.get_best_hyperparameters()[0]
"""
tuned_hp = True

# Check for tpu and then build optimally tuned model
if not tpu:
    densenet = build_densenet_model(tuned_hp)
if tpu and colab:
    with tpu_stat.scope():
        densenet = build_densenet_model(tuned_hp)

# Show summary of optimal model
densenet.summary()

## DenseNet Training

In [ ]:
# ------------------------------ WORK IN PROGRESS ------------------------------
# Set callback function for early stopping to save best epoch
callbacks = [
    ModelCheckpoint(filepath='best_densenet_har_model.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau(monitor='val_accuracy', patience=5, factor=0.1, min_lr=1.0e-7),
    EarlyStopping(monitor='val_accuracy', patience=10),
]

# Fit tuned model and save resulting weights
history = densenet.fit_generator(generator=train_dataset, validation_data=val_dataset, epochs=100, callbacks=callbacks)
densenet.save("densenet_har_model.h5")
densenet.save_weights("densenet_har_weights.h5")

# If using Colab it will save weights to local machine
if colab:
    from google.colab import files
    files.download("densenet_har_model.h5")
    files.download("densenet_har_weights.h5")

## Testing and Results

In [ ]:
# Create labels dict for reference
labels_ref = (train_dataset.class_indices)
labels = {}
for name, index in labels_ref.items():
    labels[index] = name

# Print labels dictionary evenly for a visual
print( "------------------------------" )
print( "Labels: " )
for index in range(len(labels)):
    if index < 10:
        print( " " + str(index) + ": " + labels[index] )
    else:
        print( str(index) + ": " + labels[index] )

# Collect 5 random pictures from test files and run predictions
test_files = os.listdir(TEST_DIR)
random_numbers = np.random.randint(low=0, high=len(test_files), size=5)
for i in random_numbers:
    # Collect test file and handle image
    test_file = TEST_DIR + "/" + test_files[i]
    test_image = keras.preprocessing.image.load_img(test_file, target_size=(224, 224))
    test_image = keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_file = mp.image.imread(test_file)
    plot = mp.pyplot.imshow(test_file)
    print( "------------------------------" )
    print( "\n" )
    print( test_files[i] )
    print( "\n" )
    mp.pyplot.show()
    print( "\n" )
    # Run predictions from tested model
    #prediction = cnn.predict([[test_image]])
    #prediction = vgg.predict([[test_image]])
    prediction = effnet.predict([[test_image]])
    results = {}
    for index in range(len(prediction[0])):
        results[index] = (prediction[0][index])
    sorted_results = sorted(results, key=results.get, reverse=True)
    for index in sorted_results:
        print( labels[index] + ": " + str(results[index]) )
        print( "\n" )